In [1]:
!nvidia-smi

Thu Apr  4 22:52:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:17:00.0 Off |                  Off |
| 30%   59C    P0              86W / 300W |      4MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install git+https://github.com/openai/whisper.git
!pip install pydub==0.25.1
!pip install langchain==0.1.9

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ymoym6i2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ymoym6i2
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802826 sha256=686cf487b4fae454629452e04833acc81ec2687bbdc0c5cf6fd0dac8138f3253
  Stored in directory: /tmp/pip-ephem-wheel-cache-06ihhmpv/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisp

In [3]:
import os
import torch
from pydub import AudioSegment
import whisper
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:1


In [5]:
folder_path = os.getcwd()+'/Kint_Videos'
os.listdir(folder_path)

['.ipynb_checkpoints',
 '2023-10-04_KInt_default.mp4',
 '2023-11-17_KInt_default.mp4',
 '2023-12-08_KInt_default.mp4',
 '2023-10-25_Kint_default.mp4',
 '2023-11-10_Kint_default.mp4',
 '2023-12-01_KInt_default.mp4',
 '2023-11-03_KInt_default.mp4',
 '2023-09-27_KInt_default.mp4',
 '2023-10-11_KInt_default.mp4',
 '2024-01-17_KInt_default.mp4',
 '2023-11-24_KInt_default.mp4',
 '2023-10-18_Kint_default.mp4',
 '2024-01-12_KInt_default.mp4',
 '2024-01-10_KInt_default.mp4']

In [6]:
folder_path = os.getcwd()+'/Kint_Videos'

# Iterate over files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".mp4"):
        video_path = os.path.join(folder_path, file_name)

        # Extract audio from the video
        video = AudioSegment.from_file(video_path, format="mp4")
        
        # Create a corresponding mp3 file name
        audio_output_path = os.path.join(folder_path, f"{os.path.splitext(file_name)[0]}.mp3")

        # Export audio to mp3
        video.export(audio_output_path, format="mp3")

        print(f"Audio exported for {file_name} to {audio_output_path}")

Audio exported for 2023-10-04_KInt_default.mp4 to /home/jovyan/Kint_Videos/2023-10-04_KInt_default.mp3
Audio exported for 2023-11-17_KInt_default.mp4 to /home/jovyan/Kint_Videos/2023-11-17_KInt_default.mp3
Audio exported for 2023-12-08_KInt_default.mp4 to /home/jovyan/Kint_Videos/2023-12-08_KInt_default.mp3
Audio exported for 2023-10-25_Kint_default.mp4 to /home/jovyan/Kint_Videos/2023-10-25_Kint_default.mp3
Audio exported for 2023-11-10_Kint_default.mp4 to /home/jovyan/Kint_Videos/2023-11-10_Kint_default.mp3
Audio exported for 2023-12-01_KInt_default.mp4 to /home/jovyan/Kint_Videos/2023-12-01_KInt_default.mp3
Audio exported for 2023-11-03_KInt_default.mp4 to /home/jovyan/Kint_Videos/2023-11-03_KInt_default.mp3
Audio exported for 2023-09-27_KInt_default.mp4 to /home/jovyan/Kint_Videos/2023-09-27_KInt_default.mp3
Audio exported for 2023-10-11_KInt_default.mp4 to /home/jovyan/Kint_Videos/2023-10-11_KInt_default.mp3
Audio exported for 2024-01-17_KInt_default.mp4 to /home/jovyan/Kint_Video

In [7]:
model = whisper.load_model("large")

In [8]:
folder_path = os.getcwd()+'/Kint_Videos'
segments = []

# Iterate over files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".mp3"):
        audio_path = os.path.join(folder_path, file_name)

        # Transcribe the audio
        transcription_result = model.transcribe(audio_path)

        # Append the result to the segments list along with the file name
        for segment in transcription_result['segments']:
            segment['video_name'] = os.path.splitext(file_name)[0]
            segments.append(segment)

# Now, 'result' contains the concatenated transcripts of all mp3 files in the folder
print(len(segments))

24416


In [9]:
len(segments)

24416

In [10]:
# Extracting only the desired keys from each segment
filtered_segments = [{key: segment[key] for key in ['id', 'seek', 'start', 'end', 'text', 'video_name']} for segment in segments]
len(filtered_segments)

24416

In [11]:
import json
import os
from IPython.display import display, HTML

def save_json_to_file(data, filename):
    file_path = "segments_updated.json"
    # Check if the file already exists
    if os.path.exists(file_path):
        print("file already exists")
        # Load existing JSON data from the file
        with open(file_path, "r") as json_file:
            existing_data = json.load(json_file)
    else:
        # If the file doesn't exist, initialize an empty list
        existing_data = []

    # Append the new array to the existing data
    existing_data.extend(filtered_segments)

    # Convert the combined data to JSON format
    json_data = json.dumps(existing_data, indent=4)

    # Write the JSON data back to the file
    with open(file_path, "w") as json_file:
        json_file.write(json_data)

    print(f"JSON file has been updated at {file_path}")

def generate_download_link(filename):
    download_link = f'<a href="{filename}" download>Download json file containing segments.</a>'
    display(HTML(download_link))

# Example usage:
data = filtered_segments
filename = "segments_updated.json"
save_json_to_file(data, filename)
generate_download_link(filename)

JSON file has been updated at segments_updated.json
